In [1]:
import pandas as pd
data = pd.read_csv("../Dataset/annotated_ner_data_new.csv")
data.head()

ImportError: Unable to import required dependencies:
numpy: No module named 'numpy'

In [2]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['token'].to_list()))
    else:
        vocab = list(set(data['ner'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

NameError: name 'data' is not defined

In [3]:
data['token_idx'] = data['token'].map(token2idx)
data['ner_idx'] = data['ner'].map(tag2idx)
data.head()

NameError: name 'data' is not defined

In [4]:
# Fill na
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['tweet_id'],as_index=False
)['token', 'ner', 'token_idx', 'ner_idx'].agg(lambda x: list(x))
# Visualise data
data_group.head()

ipykernel_launcher:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,tweet_id,token,ner,token_idx,ner_idx
0,1380681275594305537,"[@ridwankamil, @KickAndyShow, @Metro_TV, Upaya...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-P...","[3111, 26793, 8514, 9504, 6403, 12760, 34551, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 12,..."
1,1380685209075544064,"[@GyuuPotter, iyhh, ,, soalnya, mwu, kerjain, ...","[O, O, O, O, O, O, O, O]","[33432, 6998, 3546, 7878, 28971, 22640, 15100,...","[3, 3, 3, 3, 3, 3, 3, 3]"
2,1380685355897085952,"[@ShopeeID, Bismillah, yok, menang, ����, SHOP...","[O, O, O, O, O, B-PROD, O, O, O, O]","[28754, 32173, 3866, 4345, 20876, 23952, 28754...","[3, 3, 3, 3, 3, 7, 3, 3, 3, 3]"
3,1380686115007393792,"[udah, 2, mingguan, tinggal, bareng, teteh, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC,...","[1099, 2985, 22473, 20431, 8360, 20363, 3546, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, ..."
4,1380688640611786754,"[@ShopeeID, Potong, buah, semangka, pakai, ala...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG,...","[28754, 32875, 14209, 25392, 3580, 4995, 7750,...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 10, 3,..."


In [5]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)    
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)


In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

In [ ]:
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [ ]:

def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    #Optimiser 
    # adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [ ]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)